# Practico Redes Neuronales

Librerias a usar:
- tensorflow >= 1.13.1
- tensorboard >= 1.13.1
- keras >= 2.2.4
- Pillow >= 6.0.0
- scikit-learn >= 0.20.3

## Ejercicio 1: Entrenamiento

- Descargue y lea el paper [Using Artificial Intelligence to Predict the Equilibrated Postdialysis Blood Urea Concentration](https://drive.google.com/open?id=1TEcpoTI6mCEhvfpZv3Gp1GBr6uA0Erfg). Utilize redes neuronales para implementar el modelo propuesto en dicha publicacion. Utilice los siguientes [datos](https://drive.google.com/open?id=17bIDVoixcTlUv2JaCDRBRsU1g-WPwnBj) para el entrenamiento. Ayuda: [Tutorial Keras](https://keras.io/getting-started/sequential-model-guide/#examples).
- Si bien se presenta e intenta resolver un problema de regresion, podemos convertirlo en clasificacion utilizando Ueq > 35 como punto de corte. Genere dicha conversion de datos, asignandoles la clase 0 para Ueq < 35 y 1 en caso contrario. Entrene una red de clasificacion.

## Ejercicio 2: Extraccion de caracteristicas

- Descargue el dataset **[pets](https://drive.google.com/open?id=1JyuO-CNruuUdj14SW4s1RExBa3gMhjBi)** con imagenes de gatos y perros.
- Revise y elija dos redes convolucionales pre-entrenadas en *imagenet* de la [pagina](https://keras.io/applications/) de modelos de Keras.
- Genere dos bases de datos con vectores de caracteristicas (features) extraidos utilizando los modelos elegidos en el dataset de mascotas.
    - Tip 1: NO cargue cada imagen introduciendo a mano la ruta a el archivo. Utilice Python para iterar por todo el dataset y cargar los archivos.
    - Tip 2: Recuerde que cada modelo requiere antes un pre-procesamiento de las imagenes.
    - Tip 3: Si carga las imagenes con OpenCV estaran en formato BGR, Keras por defecto usa RGB, recuerde convertir los canales de color.
- Entrene una red neuronal para cada dataset con el objetivo de clasificar entre perros y gatos. Compare los resultados obtenidos entre ambas bases de datos en terminos de accuracy pero tambien en velocidad (que modelo de extraccion de features es mas rapido?).


In [1]:
import tensorflow as tf
import keras as ks
import tensorboard as tb 
import PIL as P
import sklearn as skelearn 
%matplotlib inline

import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib
matplotlib.rcParams['figure.figsize'] = 12, 12

Using TensorFlow backend.


In [2]:
urea = pd.read_csv('urea/urea.csv')
urea


,Upre,U120,Upos,PP,UF,Ueq,Urea
0,121,63,47,94.5,2.9,51.0,1
1,166,87,68,59.4,1.4,71.0,1
2,196,68,40,61.6,1.9,42.0,1
3,167,73,43,45.7,2.6,43.0,1
4,128,64,46,54.8,1.1,46.0,1
5,127,77,50,72.6,1.8,56.0,1
6,139,49,28,45.3,2.5,32.0,0
7,130,73,44,69.4,4.4,53.0,1
8,123,68,51,76.9,3.2,49.0,1
9,134,60,47,63.9,2.7,50.0,1


In [3]:
urea.isnull().any()

Upre    False
U120    False
Upos    False
PP      False
UF      False
Ueq     False
Urea    False
dtype: bool

In [4]:
import numpy
numpy.random.seed(7)

In [5]:
X = urea.iloc[:,0:6].values
Y = urea.iloc[:,6].values

In [10]:
# crea el modelo
from tensorflow.python.keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

TypeError: The added layer must be an instance of class Layer. Found: <keras.layers.core.Dense object at 0x7f8e3b9deac8>